In [ ]:
#imports
import pymysql
import nltk
import os
import pandas as pd
import praw
import mysql.connector

from sqlalchemy import create_engine
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

In [ ]:
#Load the variables from .env file for privicy
load_dotenv()

In [ ]:
#Gets the actual values fro, .env
id = os.getenv("client_id")
secret = os.getenv("client_secret")
pw = os.getenv("pw")
user = "Scraper 1.0 by /u/ZaaLiah"

In [ ]:
#Connection to the API
reddit = praw.Reddit(client_id=id, client_secret=secret, user_agent=user)

In [ ]:
#Search fro the trending hot posts on r/ Politics and store those in a df
df = []
for submission in reddit.subreddit('politics').hot(limit=1000):
    df.append([submission.id, submission.created_utc, submission.title, submission.num_comments, submission.score])
df = pd.DataFrame(df, columns = ['id', 'timestamp', 'title', 'num_comments', 'score'])

In [ ]:
#Converts timestamp column into only Date column and drops the old timestamp column
df['date'] = pd.to_datetime(df['timestamp'], unit='s').dt.date
df = df.drop('timestamp', axis=1)
df = df[['id', 'date', 'title', 'num_comments', 'score']]

In [ ]:
#Connects to my SQL Severs, creates a cursor and writes new posts on the DB
#Creates my fact table 'f_reddit_posts'
#If the post already exists in my DB it will update the row instead of creating a duplicate
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = pw,
    database="localdb",
    port='3306'
  )

cursor = mydb.cursor()
for row in df.itertuples():
  cursor.execute(
    'INSERT INTO f_reddit_posts (id, date, title, num_comments, score)VALUES (%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE id=VALUES (id)',
    (row.id,row.date,row.title,row.num_comments,row.score),
  row)
mydb.commit()
cursor.close()

In [ ]:
#Reads the 'f_reddit_posts' table and stores it on a df
sql = """
SELECT *
FROM f_reddit_posts
"""
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = pw,
    database="localdb",
    port='3306'
  )
df = pd.read_sql(sql, mydb)

In [ ]:
#Runs the sentiment analysis on the posts
sia = SIA()
df['compound'] = [sia.polarity_scores(x)['compound'] for x in df['title']]
df['neg'] = [sia.polarity_scores(x)['neg'] for x in df['title']]
df['neu'] = [sia.polarity_scores(x)['neu'] for x in df['title']]
df['pos'] = [sia.polarity_scores(x)['pos'] for x in df['title']]

In [ ]:
#Creates the columns with the sentiment analysis results
df['label'] = 0
df.loc[df['compound'] > 0.2, 'label'] = 1
df.loc[df['compound'] < -0.2, 'label'] = -1
df = df[['id', 'compound', 'neg', 'neu', 'pos', 'label']]

In [ ]:
#Connects to my SQL Severs, creates a cursor and writes the results on the DB
#Creates my dimension table 'd_rp_sentiment'
cursor = mydb.cursor()
for row in df.itertuples():
  cursor.execute(
    'INSERT INTO d_rp_sentiment (id, compound, neg, neu, pos, label)VALUES (%s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE id=VALUES (id)',
    (row.id,row.compound,row.neg,row.neu,row.pos,row.label),
  row)
mydb.commit()
cursor.close()